In [1]:
# !conda install plotly -y
# !conda install geopy -y
# !pip install geopy

In [1]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd

combined_data.csv  : data dictionary:

1. Job Title: Job titles for which job posting was posted: string
2. Company: Company for which job posting was posted: string
3. Location: Location for which job posting was posted: string
4. Date Posted: Date when the job was posted: string
5. Job type: 6 broad types: Software Engineer, QA Engineer, Network Engineer, Cybersecurity Analyst, Game Development, Business Analyst, Data Analyst, Data Scientist, Data Engineer, Machine Learning Engineer  : string
6. Category: CS/ DS : string

In [2]:
df = pd.read_csv('combined_data.csv')

df.head()

,Job Title,Company,Location,Date Posted,Job Type,Category
0,Analyst - Industrial Stone Operations,Tiffany & Co.,"New York, NY",11/21/2023,Business Analyst,DS
1,Business Analyst 2 - OPS Training,Huntington Bank,"Plymouth, MN",11/28/2023,Business Analyst,DS
2,Applications Analyst II,Tufts Medicine,"Burlington, MA",11/28/2023,Business Analyst,DS
3,Business Analyst (Hybrid),Secura,"Neenah, WI",11/17/2023,Business Analyst,DS
4,Business Systems Analyst,Harvard University,"Cambridge, MA",11/28/2023,Business Analyst,DS


In [3]:
# check locations
df['Location'].value_counts()

Location
Remote                632
United States         113
Washington, DC         82
Austin, TX             75
San Francisco, CA      72
                     ... 
Tremont, IL             1
Rome, NY                1
Tumwater, WA            1
Utica, NY               1
Mount Sterling, IL      1
Name: count, Length: 889, dtype: int64

We need to clean the location data. 

In [4]:
df['State'] = df['Location'].apply(lambda loc: loc.split(', ')[-1] if isinstance(loc, str) and loc not in ['United States', 'Remote'] else None)


In [5]:
invalid_states = df.loc[~df['State'].isin([
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
    'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
    'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
    'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
    'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
])]

# Print the invalid 'State' values
print(invalid_states['State'].value_counts())

State
DC                                      83
California                              26
Indiana                                 10
PR                                       9
Massachusetts                            9
Texas                                    7
Minnesota                                6
Florida                                  6
Virginia                                 4
Pennsylvania                             4
Michigan                                 4
New Jersey                               4
North Carolina                           4
Illinois                                 4
Delaware                                 4
Maryland                                 3
Georgia                                  3
Missouri                                 3
Washington State                         3
United States Minor Outlying Islands     3
Nevada                                   3
Kentucky                                 2
New Hampshire                            2
Oklah

In [6]:
state_mapping = {
    'DC': 'DC', 'California': 'CA', 'Indiana': 'IN', 'PR': 'PR', 'Massachusetts': 'MA',
    'Texas': 'TX', 'Minnesota': 'MN', 'Florida': 'FL', 'Virginia': 'VA', 'Pennsylvania': 'PA',
    'Michigan': 'MI', 'New Jersey': 'NJ', 'North Carolina': 'NC', 'Illinois': 'IL', 'Delaware': 'DE',
    'Maryland': 'MD', 'Georgia': 'GA', 'Missouri': 'MO', 'Washington State': 'WA',
    'United States Minor Outlying Islands': 'UM', 'Nevada': 'NV', 'Kentucky': 'KY',
    'New Hampshire': 'NH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'GU': 'GU', 'Colorado': 'CO', 'Idaho':'ID', 'Ohio':'OH', 'Vermont':'VT', 'Nebraska':'NE',
    'Puerto Rico': 'PR', 'Connecticut': 'CT', 'Arizona':'AR'
}

df['State'] = df['State'].apply(lambda x: state_mapping.get(x, x))

df['State'].value_counts()

State
CA    500
TX    268
VA    235
FL    154
MA    141
WA    137
NY    135
IL    131
PA    113
MD     96
NC     88
CO     87
GA     87
DC     83
MN     80
OH     80
NJ     72
MI     69
AZ     46
TN     42
MO     41
IN     41
UT     39
WI     32
NV     30
AL     24
AR     23
OK     22
SC     22
KY     22
OR     22
KS     22
CT     14
MS     13
NE     13
NH     13
ID     13
WV     10
PR     10
HI      8
DE      8
ND      8
MT      7
RI      7
ME      7
LA      6
IA      5
NM      5
UM      3
VT      3
AK      3
SD      1
WY      1
GU      1
Name: count, dtype: int64

In [7]:
df.head()

,Job Title,Company,Location,Date Posted,Job Type,Category,State
0,Analyst - Industrial Stone Operations,Tiffany & Co.,"New York, NY",11/21/2023,Business Analyst,DS,NY
1,Business Analyst 2 - OPS Training,Huntington Bank,"Plymouth, MN",11/28/2023,Business Analyst,DS,MN
2,Applications Analyst II,Tufts Medicine,"Burlington, MA",11/28/2023,Business Analyst,DS,MA
3,Business Analyst (Hybrid),Secura,"Neenah, WI",11/17/2023,Business Analyst,DS,WI
4,Business Systems Analyst,Harvard University,"Cambridge, MA",11/28/2023,Business Analyst,DS,MA


# Version 1

plot locations with respect to number of jobs (excluding remote and US)

In [16]:
import plotly.express as px


location_counts = df['State'].value_counts().reset_index()
location_counts.columns = ['State', 'Number of Jobs']


# Plotting on a US map with state boundaries
fig = px.choropleth(
    location_counts,
    locations='State',
    locationmode='USA-states',
    color='Number of Jobs',
    title='Number of Jobs in Each State (USA)',
    color_continuous_scale='viridis',
    scope='usa',
)
# 
# Show the plot
fig.show()


The scope='usa' parameter in the px.choropleth function specifies that the map should focus on the continental United States, excluding territories and surrounding islands. If you want to include territories and surrounding islands, you can use scope='north america' instead.

Plot top locations with highest number of jobs

In [11]:
# Keeping top 10 states and grouping others
top_states = location_counts.head(10)
others = pd.DataFrame({'State': ['Others'], 'Number of Jobs': [location_counts.iloc[10:].sum()['Number of Jobs']]})

# Concatenating top 5 and Others
final_state_counts = pd.concat([top_states, others])

# Plotting a pie chart
fig = px.pie(
    final_state_counts,
    names='State',
    values='Number of Jobs',
    title='Distribution of Jobs by State (Top 10 + Others)',
)

# Show the pie chart
fig.show()

The pie chart lacks clarity, and following Steven's Law, the use of area is not an effective visualization method. Choosing length over area, we will opt for a bar graph to enhance clarity and improve the visual representation.

# plot bar graph wrt number of jobs in each state

In [21]:
# Plotting with Plotly
fig = px.bar(df['State'].value_counts(), x=df['State'].value_counts().index, y=df['State'].value_counts().values,
             labels={'x': 'State', 'y': 'Number of Jobs'}, title='Number of Jobs in Each Location')

# Show the plot
fig.show()

Let's just visualise top 10 states with highest number of jobs

In [13]:
# Plotting with Plotly
fig = px.bar(df['State'].value_counts()[:10], x=df['State'].value_counts()[:10].index, y=df['State'].value_counts().values[:10],
             labels={'x': 'State', 'y': 'Number of Jobs'}, title='Top 10 Locations wrt number of jobs')

# Show the plot
fig.show()

We seee most of the CS and DS jobs are posted in CA, followed bt texas and Virginia and so on.

Let's analyse these wrt to CS and DS separately and then wrt each job type (SDE, MLE, DA, BA ...)

In [17]:
def viz(jobtype=None, category=None):

    if jobtype:
        df_new = df[df['Job Type'] == jobtype]
    if category:
        df_new = df[df['Category'] == category]
    
    # plot top 10 locations
    # Plotting with Plotly
    label = str(category) if category is not None else '' + str(jobtype) if jobtype is not None else ''
    print(label)
    fig = px.bar(df_new['State'].value_counts()[:10], x=df_new['State'].value_counts()[:10].index, y=df_new['State'].value_counts().values[:10],
                labels={'x': 'State', 'y': 'Number of Jobs'}, title='Number of Jobs in Each Location wrt '+label)

    # Show the plot
    fig.show()

    # plot choropeth map
    location_counts = df_new['State'].value_counts().reset_index()
    location_counts.columns = ['State', 'Number of Jobs']


    # Plotting on a US map with state boundaries
    fig = px.choropleth(
        location_counts,
        locations='State',
        locationmode='USA-states',
        color='Number of Jobs',
        title='Number of Jobs in Each State (USA) in domain: '+label,
        color_continuous_scale='ylorrd',
        scope='usa',
    )

    # Show the plot
    fig.show()


In [18]:
for category in df['Category'].unique():
    viz(category=category)

DS


CS


Notice the white gap? that is because there is no job posting there, handle that by having 0 as count

In [25]:
import pandas as pd
import plotly.express as px

def viz(jobtype=None, category=None):
    # Filter DataFrame based on jobtype and category
    df_filtered = df.copy()  
    if jobtype:
        df_filtered = df_filtered[df_filtered['Job Type'] == jobtype]
    if category:
        df_filtered = df_filtered[df_filtered['Category'] == category]

    # Plot top 10 locations as a bar chart
    label = str(category) if category else '' + str(jobtype) if jobtype else ''
    fig_bar = px.bar(
        df_filtered['State'].value_counts()[:10],
        x=df_filtered['State'].value_counts()[:10].index,
        y=df_filtered['State'].value_counts().values[:10],
        labels={'x': 'State', 'y': 'Number of Jobs'},
        title=f'Top 10 Locations with highers number of Jobs with respect to {label}'
    )

    # Show the bar chart
    fig_bar.show()

    # Plot choropleth map with state boundaries
    location_counts = df_filtered['State'].value_counts().reset_index()
    location_counts.columns = ['State', 'Number of Jobs']

    # Create a reference DataFrame with all states and initialize count to 0
    all_states = [
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
        'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
        'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
    ]
    reference_df = pd.DataFrame({'State': all_states, 'Count': 0})

    # Merge the reference DataFrame with location_counts
    result_df = pd.merge(reference_df, location_counts, how='left', on='State')

    # Fill NaN values with 0
    result_df['Number of Jobs'] = result_df['Number of Jobs'].fillna(0).astype(int)

    # Plot choropleth map
    fig_choropleth = px.choropleth(
        result_df,
        locations='State',
        locationmode='USA-states',
        color='Number of Jobs',
        title=f'Number of Jobs in Each State (USA) in domain: {label}',
        color_continuous_scale='ylorrd',
        scope='usa',
    )

    # Set a constant color for states with no records
    fig_choropleth.update_geos(bgcolor='lightgrey', showcoastlines=True)

    # Show the choropleth map
    fig_choropleth.show()



In [26]:
for category in df['Category'].unique():
    viz(category=category)

In [27]:
for jobtype in df['Job Type'].unique():
    viz(jobtype=jobtype)

In [28]:
remote_df = df[df['Location']=='Remote']
remote_df

,Job Title,Company,Location,Date Posted,Job Type,Category,State
42,Netsuite Business Analyst,"Universal Pure, LLC",Remote,11/03/2023,Business Analyst,DS,None
45,Eagle Pace Business Analyst,"Visvero, Inc.",Remote,11/23/2023,Business Analyst,DS,None
75,Curriculum Analyst - Business Analyst,TLN Worldwide Enterprises Inc,Remote,11/17/2023,Business Analyst,DS,None
81,Sales Strategy Analyst,Constellation Brands,Remote,11/15/2023,Business Analyst,DS,None
106,Sr Business Analyst,Rite Aid,Remote,11/14/2023,Business Analyst,DS,None
...,...,...,...,...,...,...,...
3925,Junior Software Engineer,San Data System,Remote,11/03/2023,Software Engineer,CS,None
3926,Software Engineer (.NET),The Helper Bees,Remote,11/03/2023,Software Engineer,CS,None
3927,Software Engineer (JavaScript),"Mosaic Learning, Inc.",Remote,10/28/2023,Software Engineer,CS,None
3931,Full Stack Developer,Society of Scholars,Remote,10/26/2023,Software Engineer,CS,None


In [29]:
fig = px.bar(
    remote_df['Job Type'].value_counts(),
    x=remote_df['Job Type'].value_counts().index,
    y=remote_df['Job Type'].value_counts().values,
    labels={'x': 'Job Type', 'y': 'Number of Remote Jobs'},
    title='Number of Remote Jobs Across Different Job Types'
)

# Show the plot
fig.show()

In [30]:
fig.

SyntaxError: invalid syntax (4125878239.py, line 1)

In [31]:
fig = px.bar(
    remote_df['Category'].value_counts(),
    x=remote_df['Category'].value_counts().index,
    y=remote_df['Category'].value_counts().values,
    labels={'x': 'Category', 'y': 'Number of Remote Jobs'},
    title='Number of Remote Jobs Across Different Job Categories'
)

# Show the plot
fig.show()